In [13]:
from dotenv import load_dotenv
from typing import TypedDict, Annotated
from skillkit import SkillManager
from skillkit.integrations.langchain import create_langchain_tools
from langchain.agents import create_agent
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.messages import HumanMessage, SystemMessage, AIMessage


load_dotenv()

# Discover skills
manager = SkillManager(skill_dir="./.gemini/skills")
manager.discover()

# Convert to LangChain tools
tools = create_langchain_tools(manager)


llm = ChatGoogleGenerativeAI(model="gemini-3-pro-preview")


# Create general agent
general_agent = create_agent(
    llm
    )

# create skilled agent
skilled_agent = create_agent(
    llm,
    tools=tools
    )

In [ ]:
from pydantic import BaseModel, Field
from typing import Dict, Literal

ParameterKeys = Literal[
    "tau_pv", "tau_v1", "tau_v2", "tau_pw", "tau_mw", 
    "tau_d", "tau_0", "tau_r", "tau_si", "K", 
    "V_csi", "V_c", "V_v", "C_m"
]

ToolKeys = Literal["S1-S2", "Pacing", "Plot Tip", "Save Voltage"]

class SimulationConfig(BaseModel):
    parameters: Dict[ParameterKeys, float] = Field(
        description="Updated physiological parameters based on user input."
    )
    tools: Dict[ToolKeys, bool] = Field(
        description="Boolean toggles for specific simulation tools."
    )

parameters = {
    "tau_pv": 7.99,
    "tau_v1": 9.8,
    "tau_v2": 312.5,
    "tau_pw": 870.0,
    "tau_mw": 41.0,
    "tau_d": 0.5714,
    "tau_0": 12.5,
    "tau_r": 33.83,
    "tau_si": 29.0,
    "K": 10.0,
    "V_csi": 0.861,
    "V_c": 0.13,
    "V_v": 0.04,
    "C_m": 1.0,
}

tools = {
    "S1-S2": False,
    "Pacing": False,
    "Plot Tip": False,
    "Save Voltage": False, 
}


class InputState(TypedDict):
    user_input: str

class OutputState(TypedDict):
    simulation_codes: str # file path to simulation codes
    simulation_results: str # file path to simulation results

class OverallState(TypedDict):
    user_input:str
    error: str

    parameters: dict
    tools: dict

def parameter_agent(state:InputState) -> OverallState:
    user_input = state['user_input']
    
    system_message = SystemMessage(content=(
        "You are an expert in cardiac electrophysiology and computational modeling. "
        "Your goal is to parse user requests and identify which simulation tools "
        "and parameters (like tau_pv or tau_pv) are required. "
        "Be precise and prioritize tools that enhance simulation visualization."
    ))
    
    human_message = HumanMessage(content=user_input)
    
    messages = [
        system_message, 
        human_message
    ]    
    
    result = general_agent.invoke({"messages": messages})
    return result

def model_agent(state:OverallState) -> OverallState:
    user_input = state['user_input']
    parameters = state['parameters']
    
    messages = [HumanMessage(content=user_input)]
    result = skilled_agent.invoke({"messages": messages})
    return result

def tools_agent(state:OverallState) -> OverallState:
    user_input = state['user_input']
    parameters = state['parameters']
    tools = state['tools']
    
    messages = [HumanMessage(content=user_input)]
    result = skilled_agent.invoke({"messages": messages})
    return result
